# jaxify

> Make autograd functionality compatible to JAX

In [ ]:
#| default_exp jaxify

In [ ]:
#| exporti
from ceviche import jacobian
import numpy as np
import jax
import jax.numpy as jnp

In [ ]:
#| export
def jaxit(mode='reverse', argnums=[0]):
  def inner(function):
    """
    make a function that internally uses autograd compatible to jax gradient calculations
    only a single output variable is supported
    """
    grad_fns = [jacobian(function, mode=mode, argnum=i) for i in argnums]

    @jax.custom_jvp
    def jaxed(*args):
      return function(*args)
    
    @jaxed.defjvp
    def jaxed_jvp(primals, tangents):
      primals_out = jaxed(*primals)
      as_np = [np.asarray(prim) for prim in primals]
      grads = [jnp.array(grad_fns[i](*as_np)) for i in argnums]

      # if len(tangents) > len(grads):
      #   raise RuntimeError("passed `num_args` is lower than the actual number of arguments")
      
      contributions = jnp.array([jnp.dot(grads[i],tangents[i].flatten()) for i in argnums])
  
      return primals_out, jnp.sum(contributions)
    return jaxed
  return inner


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()